In [504]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Test").setMaster("local[*]")
sc = SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Test, master=local[*]) created by __init__ at <ipython-input-1-8d78359fe3e3>:4 

In [505]:
from __future__ import print_function
import pandas as pd
from pyspark.sql import SQLContext, Row

sqlContext = SQLContext(sc)

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark import sql
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *



In [506]:
path = r"C:\Users\vrinda.b\Downloads"

In [507]:
all_incdf_path = str(path+r"\final_all_incdf_test - Copy.csv")
all_chandf_path = path+r"\all_chandf_test.csv"
all_problemdf_path = path+r"\all_problemdf_test.csv"

In [508]:
all_incdf = sc.textFile(all_incdf_path)
all_chandf = sc.textFile(all_chandf_path)
all_problemdf = sc.textFile(all_problemdf_path)
all_outagedf = None

In [509]:
#all_incdf.collect()
#all_chandf.collect()
#all_problemdf.collect()

In [510]:
#all_incdf_df = sqlContext.createDataFrame(all_incdf, ['sys_id','app_name','ci_class','cmdb_ci','sys_class_name','opened_at','major_incident','minor_incident','incident_monitoring','caused_by','contact_type','incident_category'])
#all_incdf_df.show()

In [511]:
import pyspark.sql.functions as func
all_incdf_df = sqlContext.read.csv(all_incdf_path,header = True)
all_incdf_df = all_incdf_df.withColumn("Date", func.to_date("opened_at", "dd-MM-yyyy"))
#all_incdf_df.show()
#all_incdf_df.select('Date').show()

#Dataframe Creation
all_chandf_df = sqlContext.read.csv(all_chandf_path,header = True)
all_chandf_df = all_chandf_df.withColumn("Date", func.to_date("start_date", "dd-MM-yyyy"))
all_problemdf_df = sqlContext.read.csv(all_problemdf_path,header = True)
all_probelmdf_df = all_problemdf_df.withColumn("Date", func.to_date("opened_at", "dd-MM-yyyy"))



In [512]:
history_window_lag_size = 7
prediction_window_size = 4
major_incident_column_name = 'major_incident'
target_column_name = major_incident_column_name+ str('_count')
custom_date_column_name = 'custom_date'
ci_name_column = 'app_name'

incident_date_column = 'opened_at'
change_date_column = 'start_date'
problem_date_column = 'opened_at'
outage_date_column = 'sys_created_on'

cis_list = ['xStore_invalidCI',
 'Oracle Retail Xstore Point-of-Service (POS)',
 'Triversity',
 'Point of Sale (POS) Register – TJX',
 'Windows',
 'Oracle Retail Xstore Office (Xadmin)',
 'Sortation Servers; Sorter Hardware',
 'System Control & Automation Network (SCAN) AS400',
 'Network Intelligence',
 'ServiceNow'
]   
ci_name = cis_list[0]
category = None

final_df = None

model_training = True
ci_name = cis_list[0]

In [513]:
import pyspark.sql.functions as func
def to_date_converter(input_dataset, ts_column_name):
    input_dataset = input_dataset.withColumn("Date", func.to_date(ts_column_name, "dd-MM-yyyy"))
    return input_dataset

In [514]:
def dataset_aggregation(input_dataset, agg_column_name):
    input_dataset = to_date_converter(input_dataset, agg_column_name)
    #(c) for c in df.columns
    #agg_column_list = input_dataset.select(c for c in df.columns).map(lambda r :r.getString(0)).collect.toList 
    agg_column_list = input_dataset.columns
    agg_column_list.remove(agg_column_name)
    aggregated_dataset = date_aggregator(input_dataset, agg_column_name, agg_column_list)
    return aggregated_dataset

In [515]:
def merge_datasets(left_dataset, right_dataset, left_dataset_column_name, right_dataset_column_name, join_type='outer')
    merged_dataset = left_dataset.join(right_dataset,(left_dataset.left_dataset_column_name==right_dataset.right_dataset_column_name),joinType="outer")
    merged_dataset[merged_column_name] = merged_dataset[[left_dataset_column_name,right_dataset_column_name]].apply(lambda row:row[0] if pd.notnull(row[0]) else row[1], axis=1)
    merged_dataset_columns = left_dataset.columns + right_dataset.columns
    merged_dataset_columns.remove(left_dataset_column_name)
    merged_dataset_columns.remove(right_dataset_column_name)
    merged_dataset_columns.append(merged_column_name)
    merged_dataset = merged_dataset[merged_dataset_columns]
    merged_dataset.withColumnRenamed(columns = {merged_column_name:custom_date_column_name}, inplace =True)
    return merged_dataset

In [516]:
def process_merging_dataset(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df, incident_date_column, change_date_column, problem_date_column, outage_date_column):
    JOIN = 'outer'
    incdf_gp = dataset_aggregation(all_incdf_df, incident_date_column)
    chandf_gp = dataset_aggregation(all_chandf_df, change_date_column)
    problemdf_gp = None
    outagedf_gp = None
    if(all_problemdf_df is not None):
        problemdf_gp = dataset_aggregation(all_problemdf_df, problem_date_column)
    if(all_outagedf_df is not None):
        outagedf_gp = dataset_aggregation(all_outagedf_df, outage_date_column)

    merged_dataset = merge_datasets(chandf_gp, incdf_gp, change_date_column, incident_date_column, JOIN)
    if(problemdf_gp is not None):
        merged_dataset = merge_datasets(merged_dataset, problemdf_gp, custom_date_column_name, problem_date_column, JOIN)
    if(outagedf_gp is not None):
        merged_dataset = merge_datasets(merged_dataset, outagedf_gp, custom_date_column_name, outage_date_column, JOIN)
    
    merged_dataset = merged_dataset.fillna(0)
    return merged_dataset

In [517]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
def window_values_aggregation_next(row, column_name, target_column_name, history_window_lag_size, prediction_window_size, is_training):
    sum_value = 0.0
    std_value = 0.0
    min_value = 0.0
    max_value = 0.0
    avg_slope = 0.0
    avg_slope_sign = 0.0
    result_array = np.array([])
    #result_arr =spark.createDataFrame(sc.emptyRDD(), schema)
    
    if(is_training):
        window_size = history_window_lag_size+prediction_window_size
    else:
        window_size = history_window_lag_size
    
    for i in range(0, window_size):
        
        if (column_name != target_column_name):
            if (i >= prediction_window_size) & (i < window_size):
                dataset_coulmn_name = (column_name+'_'+str(i))
                result_array = np.append(result_array, row[dataset_coulmn_name])
                #print("dataset_coulmn_name : ",dataset_coulmn_name )
                #print('Predictors loop over i: ', i)
            
        elif (column_name == target_column_name) & (is_training):
            if (i < prediction_window_size):
                dataset_coulmn_name = (column_name+'_'+str(i))
                result_array = np.append(result_array, row[dataset_coulmn_name])
                #print("dataset_coulmn_name : ",dataset_coulmn_name )
                #print('Target loop over i: ', i)
    
    if column_name != target_column_name: 
        sum_value = np.sum(result_array)
        std_value = np.std(result_array)
        min_value = np.amin(result_array)
        max_value = np.amax(result_array)
        
        start_val = result_array[0]
        for i in range(1, len(result_array)):
            avg_slope = avg_slope + ((result_array[i]-start_val)/i)
        if avg_slope > 0.0 :
            avg_slope_sign = 1.0
        elif avg_slope < 0.0 :
            avg_slope_sign = -1.0
        
        return pd.Series([sum_value, std_value, min_value, max_value, avg_slope, avg_slope_sign])
    
    elif (column_name == target_column_name) & (is_training):
        sum_value = np.sum(result_array)
        return pd.Series([sum_value])

In [518]:
def date_aggregator(input_dataset, date_column_name, agg_column_list):
    agg_dict = {}
    new_column_dict = {}
    for column_name in agg_column_list:
        agg_dict[column_name] = "sum"
        new_column_dict[column_name] = str(column_name+"_count")
        
    agg_dataset = input_dataset.groupby([date_column_name]).agg(agg_dict).withColumnRenamed(columns=new_column_dict).reset_index(drop=False, inplace=True)
    return agg_dataset

In [519]:
from pyspark.sql.functions import col
def history_window_lag_dataset(_dataset, date_column_name, history_window_lag_size):
    #input_dataset = _dataset.sort_values(by=date_column_name, ascending=True)
    input_dataset =_dataset.orderBy(date_column_name,ascending = True)
    #input_dataset_columns = input_dataset.select()
    input_dataset_columns = input_dataset.columns
    shift_dataset=input_dataset.select([date_column_name])
    for i in range(0, history_window_lag_size):
        for col in input_dataset_columns:
            w = Window().partitionBy().orderBy(col)
            if col != date_column_name:
                x = input_dataset.select("*", lag(col,1).over(w).alias(col+'_'+str(i)))
                shift_dataset = shift_dataset.join(x)
                #.shift(i).alias(col+'_'+str(i)), axis=1)
        
    return shift_dataset

In [520]:
def applying_filter_on_datasets(
        all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df, incident_date_column, 
        change_date_column, problem_date_column, outage_date_column, ci_name, category, ci_name_column):

  
    incdf = all_incdf_df.filter(col(ci_name_column).isin("ci_name") == True)#.rdd.flatMap(lambda x: x).collect()
    incdf = incdf.select([incident_date_column, "major_incident", "minor_incident", "incident_monitoring"])
    incdf = incdf.dropna()
    
    chandf = all_chandf_df.filter(col(ci_name_column).isin("ci_name") == True)#.rdd.flatMap(lambda x: x).collect()
    chandf = chandf.dropna()
    chandf = chandf.withColumn("change", lit(1))
    chandf = chandf.select([change_date_column, "change"])
        

    
    problemdf = None
    if(all_problemdf is not None):
        problemdf = all_problemdf_df.filter(col(ci_name_column).isin("ci_name") == True)#.rdd.flatMap(lambda x: x).collect()
        problemdf = problemdf.dropna()
        problemdf = problemdf.withColumn("problem", lit(1))
        problemdf = problemdf.select([problem_date_column, "problem"])
    
    outagedf = None
    if(all_outagedf is not None):
        outagedf = all_outagedf_df.filter(col(ci_name_column).isin("ci_name") == True)#.rdd.flatMap(lambda x: x).collect()
        outagedf = outagedf.dropna()
        outagedf = outagedf.withColumn("outage", lit(1))
        outagedf = outagedf.select([outage_date_column, "outage"])

    return (incdf, chandf, problemdf, outagedf)

In [521]:
def dataset_preparation(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df, incident_date_column, 
        change_date_column, problem_date_column, outage_date_column, history_window_lag_size, 
                        prediction_window_size, custom_date_column_name, target_column_name, is_training):

    merged_dataset = process_merging_dataset(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df, incident_date_column, change_date_column, problem_date_column, outage_date_column)
    
    if(is_training):
        window_size = history_window_lag_size+prediction_window_size
    else:
        window_size = history_window_lag_size
        
    lagged_dataset = history_window_lag_dataset(merged_dataset, custom_date_column_name, window_size)
    
    merged_dataset_columns = merged_dataset.columns
    shift_dataset=lagged_dataset[[custom_date_column_name]]
    #print(merged_dataset.head)
    #print(lagged_dataset.head(100))
    for col in merged_dataset_columns:
        if col != custom_date_column_name :
            if col != target_column_name:
                shift_dataset[[col, col.split('_')[0]+"_std", col.split('_')[0]+"_min", col.split('_')[0]+"_max", col.split('_')[0]+"_avg_slope", col.split('_')[0]+"_avg_slope_dirn"]] = lagged_dataset.apply(window_values_aggregation_next, args=(col, target_column_name, history_window_lag_size, prediction_window_size, is_training), axis=1)
            elif (col == target_column_name) & (is_training) :
                shift_dataset[col] = lagged_dataset.apply(window_values_aggregation_next, args=(col, target_column_name, history_window_lag_size, prediction_window_size, is_training), axis=1)
                print('Shifted data : ',target_column_name, shift_dataset[[target_column_name]].groupby([target_column_name]).count())  
                    
    print(merged_dataset.shape, shift_dataset.shape, shift_dataset.dropna().shape)
    shift_dataset = shift_dataset.dropna()

    return shift_dataset

In [522]:
from pyspark.sql.functions import when, col
def application_processing(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df, incident_date_column, change_date_column, problem_date_column, outage_date_column, ci_name, category, ci_name_column, history_window_lag_size, prediction_window_size, custom_date_column_name, target_column_name, is_training):

    outagedf = None
    incdf, chandf, problemdf, outagedf = applying_filter_on_datasets(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf, incident_date_column, change_date_column, problem_date_column, outage_date_column, ci_name, category, ci_name_column)

    prepared_dataset = dataset_preparation(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf_df,incident_date_column, change_date_column, problem_date_column, outage_date_column, history_window_lag_size, prediction_window_size, custom_date_column_name, target_column_name, is_training)
    
    prepared_dataset.app_name = ci_name
    
    if(is_training):
        prepared_dataset =prepared_dataset.withColumn('p1_flag', when(col('target_column_name') == 0,0,1))
    
    return prepared_dataset

In [523]:
final_df = application_processing(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf, incident_date_column,change_date_column, problem_date_column, outage_date_column, ci_name, category, ci_name_column, history_window_lag_size, prediction_window_size, custom_date_column_name, target_column_name,  is_training = model_training)

print("Final dataframe shape : ", final_df.shape)
print()
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print()
for i in range(1,len(cis_list)):
    ci_name = cis_list[i]
    new_app_df = application_processing(all_incdf_df, all_chandf_df, all_problemdf_df, all_outagedf, incident_date_column, change_date_column, problem_date_column, outage_date_column, ci_name, category, ci_name_column, history_window_lag_size, prediction_window_size, custom_date_column_name, target_column_name,  is_training = model_training)
    # ========================= Concate App DF ====================================
    final_df = final_df.join(new_app_df)
    print()
    print("Final dataframe shape : ", final_df.shape)
    print()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print()

AnalysisException: "cannot resolve 'sum(`Date`)' due to data type mismatch: function sum requires numeric types, not date;;\n'Aggregate [opened_at#4093], [opened_at#4093, sum(cast(caused_by#4097 as double)) AS sum(caused_by)#4275, sum(cast(sys_id#4088 as double)) AS sum(sys_id)#4276, sum(cast(contact_type#4098 as double)) AS sum(contact_type)#4277, sum(cast(major_incident#4094 as double)) AS sum(major_incident)#4278, sum(cast(cmdb_ci#4091 as double)) AS sum(cmdb_ci)#4279, sum(cast(incident_monitoring#4096 as double)) AS sum(incident_monitoring)#4280, sum(cast(incident_category#4099 as double)) AS sum(incident_category)#4281, sum(cast(app_name#4089 as double)) AS sum(app_name)#4282, sum(cast(ci_class#4090 as double)) AS sum(ci_class)#4283, sum(Date#4248) AS sum(Date)#4284, sum(cast(minor_incident#4095 as double)) AS sum(minor_incident)#4285, sum(cast(sys_class_name#4092 as double)) AS sum(sys_class_name)#4286]\n+- Project [sys_id#4088, app_name#4089, ci_class#4090, cmdb_ci#4091, sys_cla